In [92]:
import subprocess
def get_html_from_url(url):
    cmds = ["/Applications/Google Chrome.app/Contents/MacOS/Google Chrome", "--headless", "--dump-dom", "--enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb"]
    cmds.append(url)
    result = subprocess.run(cmds, stdout=subprocess.PIPE)
    text = result.stdout.decode('utf-8')
    return text
    
# t = get_html_from_url("https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:systems-of-equations/x2f8bb11595b61c86:introduction-to-systems-of-equations/v/trolls-tolls-and-systems-of-equation")
# t
# # result = subprocess.run(['pwd'], stdout=subprocess.PIPE)
# # text = result.stdout.decode('utf-8')
# # text


In [96]:
# unit_url = "https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:systems-of-equations/x2f8bb11595b61c86:introduction-to-systems-of-equations"
unit_url = "https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:sequences"
site = "https://www.khanacademy.org"

In [97]:
from xml.dom.minidom import parseString

html = get_html_from_url(unit_url)



In [98]:
from html.parser import HTMLParser
class Parser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.href = ''
        self.urls = []

    def handle_starttag(self, tag, attrs):
        if (tag == 'a'):
            for (k,v) in attrs:
                if (k == 'href'):
                    self.href = v
        elif (tag == 'span'):
            for (k,v) in attrs:
                if (k == 'aria-label' and v == 'Video'):
                    self.urls.append(self.href)            



parser = Parser()
parser.feed(html)
parser.urls


['/math/algebra/x2f8bb11595b61c86:sequences/x2f8bb11595b61c86:introduction-to-arithmetic-sequences/v/explicit-and-recursive-definitions-of-sequences',
 '/math/algebra/x2f8bb11595b61c86:sequences/x2f8bb11595b61c86:introduction-to-arithmetic-sequences/v/arithmetic-sequences',
 '/math/algebra/x2f8bb11595b61c86:sequences/x2f8bb11595b61c86:introduction-to-arithmetic-sequences/v/extending-arithmetic-sequences',
 '/math/algebra/x2f8bb11595b61c86:sequences/x2f8bb11595b61c86:introduction-to-arithmetic-sequences/v/using-arithmetic-sequences-formulas',
 '/math/algebra/x2f8bb11595b61c86:sequences/x2f8bb11595b61c86:introduction-to-arithmetic-sequences/v/using-recursive-formula-for-arithmetic-sequence',
 '/math/algebra/x2f8bb11595b61c86:sequences/x2f8bb11595b61c86:constructing-arithmetic-sequences/v/recursive-formula-for-arithmetic-sequence',
 '/math/algebra/x2f8bb11595b61c86:sequences/x2f8bb11595b61c86:constructing-arithmetic-sequences/v/explicit-formulas-for-arithmetic-sequences',
 '/math/algebra/

In [32]:
video_url = site + parser.urls[0]
video_html = get_html_from_url(video_url)




[0521/115618.386214:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.
[0521/115618.527860:WARNING:runtime_features.cc(730)] AttributionReportingCrossAppWeb cannot be enabled in this configuration. Use --enable-features=ConversionMeasurement,AttributionReportingCrossAppWeb in addition.


In [49]:
class VideoParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.start_h1 = False
        self.start_div = False
        self.start_iframe = False
        
        self.name = ''
        self.desc = ''

    def handle_starttag(self, tag, attrs):
        if (tag == 'h1'):
            self.start_h1 = True       
        elif (tag == 'div'):
            for (k,v) in attrs:
                if (k == 'id' and v == 'ka-videoPageTabs-tabbedpanel-content'):
                    self.start_div = True
        elif (tag == 'iframe'):
            print(attrs)
            for (k,v) in attrs:
                if(k == 'data-youtubeid'):
                    print(v)

    def handle_endtag(self, tag):
        if (tag == 'h1'):
            self.start_h1 = False
        elif (tag == 'div'):
            self.start_div = False
            
    def handle_data(self, data):
        if self.start_h1:
            self.name = data.title()
        if self.start_div:
            self.desc += data

video_parser = VideoParser()
video_parser.feed(video_html)
print(video_parser.name, video_parser.desc)




[('src', 'https://www.googletagmanager.com/ns.html?id=GTM-WSX5PBP'), ('height', '0'), ('width', '0'), ('style', 'display:none;visibility:hidden')]
Systems Of Equations: Trolls, Tolls (1 Of 2) A troll forces us to use algebra to figure out the make-up of his currency. We end up setting up a system of equations. Created by Sal Khan.


In [78]:
def get_searched_value(video_html, searchStr):
    x = video_html.find(searchStr)
    start = video_html.find('"', x+len(searchStr))
    end = video_html.find('"', start+1)
    return video_html[start+1:end]


def get_topic(video_html):
    desc =  get_searched_value(video_html, '"translatedDescription":')
    name = get_searched_value(video_html, '"translatedTitle":').title()
    videoID = get_searched_value(video_html, '"translatedYoutubeId":')
    return {
        "name": name,
        "description": desc,
        "texts": ["text1.md"],
        "video_parts": [{
            "id": videoID
        }],
        "node_type": "lecture",
        "parent_id": "khan-algebra-1",
        "thumbnail": ""
    }

In [ ]:

topics = []
for url in parser.urls:
    video_url = site + url
    video_html = get_html_from_url(video_url)
    topic = get_topic(video_html)
    print(json.dumps(topic, indent=4))
    topics.append(topic)
    


    
    
    
    

{
    "name": "Sequences Intro",
    "description": "Sequences are ordered lists of numbers (called \\",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "KRFiAlo7t1E"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
{
    "name": "Intro To Arithmetic Sequences",
    "description": "Sal introduces arithmetic sequences and their main features, the initial term and the common difference. He gives various examples of such sequences, defined explicitly and recursively.",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "_cooC3yG_p0"
        }
    ],
    "node_type": "lecture",
    "parent_id": "khan-algebra-1",
    "thumbnail": ""
}
